# Boston Weather

# Setup
First, import what we need.

In [1]:
import math
import altair as alt
from altair import datum
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from vega_datasets import data

Then, check versions of installed packages.

In [2]:
'ipywidgets ' + widgets.__version__ + ', altair ' + alt.__version__ + ', pandas ' + pd.__version__ + ', numpy ' + np.__version__

'ipywidgets 8.1.1, altair 5.1.2, pandas 2.0.3, numpy 1.26.0'

Allow Altair to use more than 5000 rows

In [3]:
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

## Weather

In [4]:
bostonWeatherDF = pd.read_csv('boston-weather-mid.csv')
bostonWeatherDF

,year,month,maxTemp,minTemp,meanTemp,precipitation,CDDBase65
0,1872,Jan,NaN,NaN,NaN,2.0,NaN
1,1872,Feb,NaN,NaN,NaN,2.0,NaN
2,1872,Mar,48.0,-8.0,18.0,4.0,0.0
3,1872,Apr,85.0,28.0,38.0,1.0,1.0
4,1872,May,85.0,40.0,51.0,3.0,16.0
...,...,...,...,...,...,...,...
1816,2023,May,86.0,42.0,51.0,2.0,39.0
1817,2023,Jun,84.0,47.0,59.0,3.0,106.0
1818,2023,Jul,91.0,61.0,68.0,10.0,332.0
1819,2023,Aug,88.0,59.0,65.0,6.0,216.0


Data from [Weather.gov NOWData](https://www.weather.gov/wrh/Climate?wfo=box). Extracted monthly summary data `por`–`2023`, replaced "T" and "M" with blank for the missing data, loaded in PowerQuery to unpivot months then to pivot measure.

In [5]:
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
yearsInData=[bostonWeatherDF['year'].min(), bostonWeatherDF['year'].max()]
tempsInData=[bostonWeatherDF['meanTemp'].min(), bostonWeatherDF['meanTemp'].max()]

In [6]:
@interact(years=widgets.IntRangeSlider(
    value=yearsInData,
    min=yearsInData[0],
    max=yearsInData[1],
    step=1,
    description='Years:'
))
def tempMonthChangeOverTimeChart(years):    
    return alt.Chart(
        bostonWeatherDF, 
        title=alt.Title(
            'How the Boston temperatures for each month have changed over time',
            subtitle='A comparison of ' + str(years[0]) + '–' + str(years[1])
        )
    ).transform_filter(
        ((datum.year >= years[0]) & (datum.year <= years[1]))
    ).encode(
        color=alt.Color('year', scale=alt.Scale(scheme='lightgreyred', domain=yearsInData), sort='ascending')
    ).mark_line(point=True).encode(
        x=alt.X('month:O').sort(monthOrder),
        y=alt.Y('meanTemp:Q',  title='mean monthly temp in ⁰F', scale=alt.Scale(domain=tempsInData)),
        tooltip=['year', 'meanTemp'],
        opacity=alt.value(.3)
    ).properties(
        width=600,
        height=600
    )

interactive(children=(IntRangeSlider(value=(1872, 2023), description='Years:', max=2023, min=1872), Output()),…

In [7]:
@interact(month=widgets.Dropdown(
    options=monthOrder,
    value='Aug',
    description='Month:',
))
def tempYearChangeOverTimeChart(month):

    brush = alt.selection_interval(encodings=['x'])
    
    colorChart = alt.Chart(
        title=alt.Title(
            'How the Boston ' + month + ' temperature has changed over time',
            subtitle='A comparison of ' + str(yearsInData[0]) + '–' + str(yearsInData[1]) + ' with interactive selection mean across years'
        )
    ).encode(
        color=alt.Color('year', scale=alt.Scale(scheme='lightgreyred')),
    ).add_params(
        brush
    )

    points = colorChart.mark_point().encode(
        x=alt.X('year:Q', scale=alt.Scale(zero=False)),
        y=alt.Y('meanTemp:Q', title='mean monthly temp in ⁰F', scale=alt.Scale(zero=False)),
        tooltip=['year', 'maxTemp', 'meanTemp', 'minTemp', 'CDDBase65']        
    )

    meanLine = alt.Chart().mark_rule(color='firebrick').encode(
        y=alt.Y('mean(meanTemp):Q', scale=alt.Scale(zero=False)),
        size=alt.SizeValue(3)
    ).transform_filter(
        brush
    )

    tempScatter = alt.layer(points, meanLine, data=bostonWeatherDF).transform_filter(
        (datum.month == month)
    )
    
    return tempScatter.properties(
        width=600,
        height=300
    )

interactive(children=(Dropdown(description='Month:', index=7, options=('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun…

In [8]:
@interact(month=widgets.Dropdown(
    options=monthOrder,
    value='Aug',
    description='Month:',
))
def tempScatter(month):

    
    colorChart = alt.Chart(
        bostonWeatherDF, 
        title=alt.Title(
            'Cooling Degree Days Base 65 score for ' + month + ' over the years',
            subtitle="A comparison of 1872–2023"
        )
    ).transform_filter(
        (datum.month == month)
    ).encode(
        color=alt.Color('year', scale=alt.Scale(scheme='lightgreyred'))
    )
    
    lines = colorChart.mark_point().encode(
        x=alt.X('year:Q', scale=alt.Scale(zero=False)),
        y=alt.Y('CDDBase65:Q', scale=alt.Scale(zero=False)),
        tooltip=['year', 'maxTemp', 'meanTemp', 'minTemp', 'CDDBase65']        
    )

    tempScatter=lines.properties(
        width=600,
        height=300
    )

    return tempScatter

interactive(children=(Dropdown(description='Month:', index=7, options=('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun…